# Example of code for custom optimisation

This notebook shows how to configure and run a customised configuration for the solver

In [ ]:
from pathlib import Path

import negdis

Using `cerv` dataset as an example:

In [ ]:
DATA_DIR = Path('data', 'cerv')

POS_LOG_PATH = DATA_DIR.joinpath('full', 'cerv.pos.xes')
NEG_LOG_PATH = DATA_DIR.joinpath('full', 'cerv.neg.xes')

DECLARE_RULES_PATH = DATA_DIR.joinpath('declare_rules.txt')

Input files can also be created on the fly, in this case the set of Declare patterns are defined directly in this notebook as a string. See below on how to use it (note the use of `r''` Python construct to avoid string interpolation). Strings can be passed as file arguments by using the `negdis.as_file` context function (see below for its usage).

In [ ]:
DECLARE_PATTERNS_STR = r'''
Absence(a):[^a]*
Absence2(a):[^a]*(a)?[^a]*
Absence3(a):[^a]*((a)?[^a]*){2}
AlternatePrecedence(a,b):[^b]*(a[^b]*b[^b]*)*[^b]*
Condition(a,b):[^b]*(a.*b)*[^b]*
Existence(a):.*a.*
NotSuccession(a,b):[^a]*(a[^b]*)*[^ab]*
Response(a,b):[^a]*(a.*b)*[^a]*
'''

## Choices generation stage

Run `negdis` to generate the candidates, output is stored in a temporary file. *Negdis* executable is wrapped in the `negdis.Negdis` class; its `default` method returns an object using the file in the `dist` directory.

In [ ]:
negdis_exe = negdis.Negdis.default()
print(negdis_exe.version())

In [ ]:
import atexit
import tempfile

out_dir = Path(tempfile.mkdtemp())

@atexit.register
def cleanup():
    import shutil
    # remove the temporary directory on termination
    shutil.rmtree(out_dir)


# output on a temporary file
CHOICES_PATH = Path(out_dir).joinpath('choices.json')

with negdis.as_file(DECLARE_PATTERNS_STR) as patterns:
    negdis_exe.discover(POS_LOG_PATH, NEG_LOG_PATH, patterns, CHOICES_PATH)

print(f'Choices written to: {CHOICES_PATH}')

### Shows top constraints

In [ ]:
negdis.count_choices(CHOICES_PATH)

## Optimisation stage

The configuration of the solver can be customised using the class `negdis.SolverConf`:

In [ ]:
opt_code = r'''
#preference(p1,less(cardinality)){ ${predicate_holds}(C)}.
#preference(p2,less(cardinality)){ ${predicate_selected}(C)}.

#preference(p10,lexico){ 1::**p2; 2::**p1 }.
#optimize(p10).
'''

opt_mode = negdis.SolverConf(
    id='asprinminclos',
    inputs=['guess.lp'],
    args=['--quiet=1'],
    docstring='Minimal cardinality wrt the closure constraints, and selected constraints for ties',
    solver='asprin',
    template=opt_code
)

Actual ASP program is generated by concatenating the content of files in `inputs` with the string `template`, and then replacing the values of `${...}` macros. Default macros are:

- `predicate_holds`
- `predicate_action`
- `predicate_choice`
- `predicate_selected`
- `predicate_constraint`

but new macros can be defined (or the default value replaced) by means of a dictionary that can be passed as an argument.

If the file names in `inputs` are not absolute paths, they'll be looked up in the current directory followed by the directory of the Python package `negdis.templates`; e.g. the `guess.lp` is fetched from `./negdis/templates/guess.lp`.

The generated ASP program can be inspected using the `program` method, which accepts an *optional* argument with the mapping for the macros:

In [ ]:
print(opt_mode.program(mapping={'predicate_holds': 'closure'}))

*asprin* solver doesn't support JSON output, so for the time being only the raw solver output is printed and not statistics are reported.

In [ ]:
negdis.optimise_choices(CHOICES_PATH, opt_mode, DECLARE_RULES_PATH, models=10, timeout=60)

In [ ]:
negdis.optimise_choices(CHOICES_PATH, 'minclos', DECLARE_RULES_PATH, models=10, timeout=60)

For debugging, the ASP program can be generated using the `negdis.asp_program` function:

In [ ]:
print(negdis.asp_program(CHOICES_PATH, opt_mode, DECLARE_RULES_PATH))

In [ ]:
import shutil

shutil.rmtree(str(out_dir))